In [1]:
from process_csv import merge_gene_data_and_coords
import os
import pandas as pd
from data_loader import get_patient_loader
import torch
from model import get_res50, get_res50_dropout, get_res18_ciga, get_vgg13, get_res18, get_res18_dropout, get_res18_ciga_dropout, get_res18_1000
import torch.nn as nn
import torchvision

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

data_dir_test = "../Test_Data/"
data_dir_train = "../Training_Data/"
patients_test  = [os.path.basename(f) for f in os.scandir(data_dir_test) if f.is_dir()]
patients_train = [os.path.basename(f) for f in os.scandir(data_dir_train) if f.is_dir()]
print(patients_test)
print(patients_train)

results_dir = "../results/res50/"
gene = "RUBCNL"

['p021', 'p026', 'p008']
['p020', 'p016', 'p014', 'p013', 'p025', 'p007', 'p009']


In [21]:
raw = []
raw.append(("../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt", get_res50))
raw.append(("../models/res50/RUBCNL_HLR_Res50slim_ice/15072024_ep_39_lr_0.0005resnet.pt", get_res50))
raw.append(("../models/res50/RUBCNL_HLR_Res50slim_optim_ice/RUBCNL_HLR_Res50slim_optim_ice15072024_ep_26_lr_0.0005resnet.pt", get_res50))
raw.append(("../models/res50/RUBCNL_HLR_Res50_Drop/24072024_ep_29_lr_0.0005resnet.pt", get_res50_dropout))
raw.append(("../models/res18/res18_iced_e29.pt",get_res18_ciga))
raw.append(("../models/res18/res18_not_iced_e29.pt", get_res18_ciga))
raw.append(("../remote_models/res50/RUBCNL_Res50/MyNet2_ep_29.pt", get_res50))
raw.append(("../remote_models/res50/RUBCNL_Res50_freeze/Res50Dropout_ep_29.pt", get_res50_dropout))
raw.append(("../remote_models/res50/RUBCNL_Res50_drop_freeze/Res50Dropout_ep_29.pt", get_res50_dropout))
raw.append(("../remote_models/vgg13/dropout/VGG13_ep_29.pt", get_vgg13))
raw.append(("../remote_models/res18/RUBCNL_Res18/Res18_ep_29.pt", get_res18_1000))
raw.append(("../remote_models/res18/RUBCNL_Res18_freeze/Res18_ep_29.pt", get_res18_1000))
raw.append(("../remote_models/res18/RUBCNL_Res18_ciga/Res18_ep_29.pt", get_res18_ciga))
raw.append(("../remote_models/res18/RUBCNL_Res18_ciga_drop/Res18Dropout_ep_29.pt", get_res18_ciga_dropout))
raw.append(("../remote_models/res18/RUBCNL_Res18_ciga_freeze_drop/Res18Dropout_ep_29.pt", get_res18_ciga_dropout))


models = []
for r in raw:
    #print(r[0])
    models.append((r[1](r[0]).to(device).eval(), r[0]))

<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>


In [19]:
"""
for patient in patients_test:
    merge_gene_data_and_coords(data_dir_test,patient, results_dir, gene + "_results.csv", gene)
    
for patient in patients_train:
    merge_gene_data_and_coords(data_dir_train,patient, results_dir, gene + "_results.csv", gene)
"""

'\nfor patient in patients_test:\n    merge_gene_data_and_coords(data_dir_test,patient, results_dir, gene + "_results.csv", gene)\n    \nfor patient in patients_train:\n    merge_gene_data_and_coords(data_dir_train,patient, results_dir, gene + "_results.csv", gene)\n'

In [25]:
def generate_results2(model, model_path, device, data_dir, results_dir, patient=None, gene="RUBCNL", results_filename=None):#
    if results_filename is None:
        results_filename = gene + "_results.csv"
    model.eval()
    loader = get_patient_loader(data_dir, patient, gene)
    filename = results_dir + "/" + patient + "/" + results_filename
    if os.path.exists(filename):
        file_df = pd.read_csv(filename)
        if model_path in file_df:
            print(model_path + " already calculated")
            return
    else:
        print(filename + " does not exist")
        return

    i = 0
    j = len(loader)
    with torch.no_grad():
        for images, _, name in loader:
            if i % 1000 == 0:
                print(i, "/", j)
            i += 1
            images = images.unsqueeze(0).to(device)
            images = images.float()

            output = model(images)
            output = output.squeeze().unsqueeze(0)
            file_df.loc[file_df['tile'] == os.path.basename(name),[model_path]] = output.item()

    file_df.to_csv(filename)

for model in models:
    for patient in patients_train:
        print(model[1],patient)
        generate_results2(model[0], model[1], device, data_dir_train, results_dir, patient)
        
for model in models:
    for patient in patients_test:
        if patient == "p021":
            continue
        print(model[1],patient)
        generate_results2(model[0], model[1], device, data_dir_test, results_dir, patient)

../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p020
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p016
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p014
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p013
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p025
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt p007
../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt already calculated
../m